<a href="https://www.kaggle.com/code/valentindefour/tps-s03e03-eda-automl-fun-explainable-ai?scriptVersionId=117168272" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# IMPORTING LIBRARIES

In [1]:
import numpy as np
import pandas as pd

import optuna

import catboost as cb
from sklearn import model_selection, metrics, linear_model, ensemble, naive_bayes, neighbors, svm, tree
from xgboost import XGBClassifier
import lightgbm

import os
import gc
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s3e3/sample_submission.csv
/kaggle/input/playground-series-s3e3/train.csv
/kaggle/input/playground-series-s3e3/test.csv
/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv


# LOADING COMPETITION DATA + HISTORIC DATA FROM ORIGINAL DATASET

In [2]:
train = pd.read_csv(r'/kaggle/input/playground-series-s3e3/train.csv')
train_o = pd.read_csv(r'/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')
test = pd.read_csv(r'/kaggle/input/playground-series-s3e3/test.csv')
ss = pd.read_csv(r'/kaggle/input/playground-series-s3e3/sample_submission.csv')

# EDA

In [3]:
print(train.shape, train_o.shape, test.shape)

(1677, 35) (1470, 35) (1119, 34)


In [4]:
train.isnull().sum()

id                          0
Age                         0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithC

In [5]:
train_o.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [6]:
display(train.head())
display(train_o.head())

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,36,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,...,80,1,10,2,3,10,0,7,8,0
1,1,35,Travel_Rarely,921,Sales,8,3,Other,1,1,...,80,1,4,3,3,4,2,0,3,0
2,2,32,Travel_Rarely,718,Sales,26,3,Marketing,1,3,...,80,2,4,3,3,3,2,1,2,0
3,3,38,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,...,80,0,15,1,1,6,0,0,2,0
4,4,50,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,...,80,0,31,0,3,31,14,4,10,1


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [7]:
display(train['Attrition'].value_counts())
display(train_o['Attrition'].value_counts())

0    1477
1     200
Name: Attrition, dtype: int64

No     1233
Yes     237
Name: Attrition, dtype: int64

In [8]:
train_o['Attrition'] = train_o['Attrition'].map(lambda x: 1 if x == "Yes" else 0)
train_o['id'] = train_o['EmployeeNumber']

In [9]:
train_all = pd.concat([train, train_o.drop('EmployeeNumber', axis = 1)])

In [10]:
display(train_all.shape)
display(train_all['Attrition'].value_counts())

(3147, 35)

0    2710
1     437
Name: Attrition, dtype: int64

In [11]:
target = train_all['Attrition']

data = pd.concat([train_all.drop('Attrition', axis = 1), test]).reset_index(drop = True)

print(train_all.shape, test.shape, data.shape)

(3147, 35) (1119, 34) (4266, 34)


In [12]:
text_features = []

for column in data.columns:
    if data[column].dtype == 'object':
        text_features.append(column)

for text_feature in text_features:
    data = data.join(pd.get_dummies(data[text_feature], prefix=text_feature))
    data = data.drop(text_feature, axis = 1)

In [13]:
data.head()

,id,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0,36,599,24,3,1,4,42,3,1,...,0,0,0,0,0,1,0,1,0,1
1,1,35,921,8,3,1,1,46,3,1,...,0,0,0,1,0,1,0,1,1,0
2,2,32,718,26,3,1,3,80,3,2,...,0,0,1,0,1,0,0,1,1,0
3,3,38,1488,2,3,1,3,40,3,2,...,0,0,0,0,0,1,0,1,1,0
4,4,50,1017,5,4,1,2,37,3,5,...,0,0,0,0,0,0,1,1,0,1


In [14]:
df_train = data.iloc[:len(target), :]

df_test = data.iloc[len(target):, :]

df_train.shape, target.shape, df_test.shape

((3147, 55), (3147,), (1119, 55))

# "MANUAL" ML

In [15]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_train.drop('id', axis = 1), target, random_state = 42)

### CATBOOST

In [16]:
parameters = {'depth':[4,5,6,7,8,9, 10],
              'learning_rate':[0.01,0.02,0.03,0.04],
              'iterations':[10,20,30,40,50,60,70,80,90,100]}

In [17]:
# gs_cb = model_selection.GridSearchCV(estimator=cb.CatBoostClassifier(), param_grid=parameters, verbose=0, cv = 3, n_jobs = -1)

# gs_cb.fit(X_train, y_train)

In [19]:
model_cb = cb.CatBoostClassifier(depth = 7,
                                 learning_rate = 0.04,
                                 rsm = 0.5,
                                 iterations = 100,
                                 random_seed = 42,
                                 verbose = False).fit(X_train, y_train)

print(metrics.roc_auc_score(model_cb.predict(X_test), y_test))
preds_cb = model_cb.predict(df_test.drop('id', axis = 1))
proba_cb = model_cb.predict_proba(df_test.drop('id', axis = 1))

0.8565609863496257


### XGBOOST

In [24]:
clf = XGBClassifier(eval_metric = 'auc')

params_xgb = {#'learning_rate': [0.1, 0.05, 0.03],
          'n_estimators': np.arange(100, 500,50)#,
#          'max_depth': [3,4,5],
#          'lambda': [1,2,3,4,5],
#          'colsample_bytree': [0.3,0.4,0.5,0.6,0.8],
#          'subsample': [0.3,0.4,0.5,0.6,0.8],
#          'scale_pos_weight': [4,5,6.2,7]
}

gs_xgb = model_selection.GridSearchCV(clf, params_xgb, n_jobs = -1, cv = 3, scoring = 'roc_auc',
                       verbose = 1)
gs_xgb.fit(X_train, y_train)
print(gs_xgb.best_score_)
print(gs_xgb.best_params_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
0.8310395128279658
{'n_estimators': 100}


In [25]:
model_xgb = XGBClassifier(subsample = 0.8, scale_pos_weight = 5 , max_depth = 3, learning_rate = 0.1, colsample_bytree = 0.8, n_estimators = 100, eval_metric = "auc").fit(X_train, y_train)

print(metrics.roc_auc_score(model_xgb.predict(X_test), y_test))

proba_xgb = model_cb.predict_proba(df_test.drop('id', axis = 1))

0.6972260195944407


### LIGHTGBM

In [26]:
model_lgbm = lightgbm.LGBMClassifier().fit(X_train, y_train)
print(metrics.roc_auc_score(model_lgbm.predict(X_test), y_test))
preds_lgbm = model_lgbm.predict(df_test.drop('id', axis = 1))
proba_lgbm = model_lgbm.predict_proba(df_test.drop('id', axis = 1))

0.7554751430868927


### BLENDING THEM ALL TOGETHER

In [28]:
proba_global = (proba_cb+proba_xgb+proba_lgbm)/3
proba_global

array([[0.7293688 , 0.2706312 ],
       [0.89515847, 0.10484153],
       [0.91225708, 0.08774292],
       ...,
       [0.94373508, 0.05626492],
       [0.96070945, 0.03929055],
       [0.9866698 , 0.0133302 ]])

### TRYING A SUBMISSION

In [31]:
ss[['Stay','Attrition']] = proba_global
ss[['id','Attrition']].to_csv('submission.csv', index = False)

This submission got a public score of
## 0.92514.

---

### Not bad, but let's now try with h2o Auto ML :

# AUTO ML

## Init cluster and data loading

In [ ]:
import h2o        
from h2o.automl import H2OAutoML

In [ ]:
h2o.init()

In [ ]:
target.to_frame().reset_index(drop = True)

In [ ]:
df_train_h2o = df_train.copy()
df_train_h2o['Attrition'] = target.to_frame().reset_index(drop = True)
train_h2o = h2o.H2OFrame(df_train_h2o)

In [ ]:
test_h2o = h2o.H2OFrame(df_test)

## Training

In [ ]:
train, test = train_h2o.split_frame(ratios = [0.75])

x = train.columns
y = 'Attrition'
x.remove(y)

In [ ]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models - or 10 min
aml = H2OAutoML(max_models=20,
                max_runtime_secs=600,
                seed=42)

aml.train(x=x, y=y, training_frame=train)

In [ ]:
lb = aml.leaderboard
lb.head()

In [ ]:
preds = aml.leader.predict(test_h2o)

In [ ]:
ss['Attrition'] = preds.as_data_frame()['p1']
ss.to_csv('submission.csv', index = False)